In [ ]:
!pip install transformers torch numpy

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.7 MB/s eta 0:00:00


In [ ]:
import xml.etree.ElementTree as ET
import re

# get the inclusion criteria of the trials:
def get_inclusion_criteria(trial,n = 0):
    """get only inclusion criteria from xml clinical trial. Input should be in xml format
   parameter n serves as a enumerator for the different trials"""
    try:
      tree = ET.parse(trial)
      root = tree.getroot()
    except Exception as e:
      print(f'trial {trial} not found')
      with open('errors.txt', 'a') as f:
         f.write(f'trial {trial}: {e}')
      return False
    #f = open('inclusion_criteria.txt', 'a')
    #f.write(f'\n\n Trial{n}\n')

    for field in root.iter('Field'):
      if field.attrib['Name'] == 'EligibilityCriteria':
          try:
            inclusion1 = re.search(r'Inclusion Criteria:(.*?)Exclusion Criteria', field.text, re.DOTALL).group(1)
            #f.write(inclusion1)
            #print(f'trial {n} inclusion criteria added')
          except AttributeError:
             print(f'Trial {trial}, problem with inclusion criteria text')
      if field.attrib['Name'] == 'HealthyVolunteers':
          #inclusion2 = re.search(r'Inclusion Criteria:(.*?)Exclusion Criteria:.*?(Healthy Volunteers:.*?$)', field.text, re.DOTALL).group(2)
          #f.write(f'Healthy Volunteers: {field.text}\n')
          healthy_volunteers = field.text
          inclusion1 = inclusion1+' '+f'Healthy Volunteers: {healthy_volunteers};'
      if field.attrib['Name'] == 'Gender':
          #f.write(f'Gender: {field.text}\n')
          gender = field.text
          inclusion1 = inclusion1+' '+f'Gender: {gender};'
      if field.attrib['Name'] == 'MinimumAge':
          #f.write(f'Minimum Age: {field.text}\n')
          min_age = field.text
          inclusion1+=f' Minimum Age: {min_age};'
      if field.attrib['Name'] == 'MaximumAge':
          #f.write(f'Maximum Age: {field.text}\n')
          max_age = field.text
          inclusion1+=f' Maximum Age: {max_age}.'
    if len(inclusion1) > 1:
    #f.close()
      return inclusion1
    else:
       return False


# uncomment to get exclusion criteria in a text file
def get_exclusion_criteria(trial, n = 0):
   """get only exclusion criteria from xml clinical trial. Input should be in XML format
   n serves as an enurmerator for the different trials"""
   try:
     tree = ET.parse(trial)
     root = tree.getroot()
   except Exception as e:
      print(f'trial {trial} not found')
      with open('errors.txt', 'a') as f:
         f.write(f'trial {trial}: {e}')
      return False
   #f = open('exclusion_criteria.txt', 'a')
   #f.write(f'\n\n Trial {n}\n')
   for field in root.iter('Field'):
      if field.attrib['Name'] == 'EligibilityCriteria':
        try:
          exclusion = re.search(r'Exclusion Criteria:(.*?$)', field.text, re.DOTALL).group(1)
          #f.write(exclusion)
          #print(f'Trial {n} criteria added')
        except AttributeError:
           print(f'trial {trial} got problem with exclusion criteria text')
   #f.close()
   if len(exclusion) > 1:
    return exclusion
   else:
      return False


In [ ]:
#path = '/content/'
trial_xml_files = ['LM_101_Injection.xml', 'Prostate_cancer.xml', 'TetraHertz.xml', 'Topical_Tazarotene.xml']
# trial_xml_files = ['BMS86015_drug.xml', 'Breast_cancer.xml', 'Maladaptive_tissue.xml', 'Neuroblastoma.xml']


In [ ]:
# list of approximately 10 different prompts, they should be applied to the 6 different trials
# Fairooz
# prompt1 = "here is a text with inclusion criteria for a clinical trial. Provide an RDF representation of this text. Make use of properties from the schema.org vocabulary, like hasInclusionCriteria, MinimumAgeRequirement, MedicalCondition criterionGroup/hasItem, LifeExpectancyRequirement, EvaluableLesionRequirement, LaboratoryExaminationRequirement, ContraceptiveUseRequirement, CommunicationRequirement, and UnderstandingRequirement. You can invent new classes where necessary."
# prompt2 = "here is a text with inclusion criteria for a clinical trial. Convert the text into RDF triples. Make sure to break down sentences if they contain more than one important information."
# Ahana
# prompt3 = "here is a text with inclusion criteria for a clinical trial. Create a triple using NCIT ontology to intricate associations among clinical trial phases, inclusion and exclusion criteria, participant demographics, and trial outcomes. Define concepts and their interconnections in the RDF graph."
# prompt4 = "here is a text with inclusion criteria for a clinical trial. Extract from this text all relevant information and present them in triples (entity1, relation, entity2), where entity1 and entity2 should be taken directly from the trial text, and relation should indicate how the to entities are connected to each other. The triples will then be used to build a knowledge graph."
# Efua
# prompt5 = "here is a text with inclusion criteria for a clinical trial. Create an RDF graph that uses triples to represent the relationship between enrollment criteria, participant characteristics and trial outcomes. Make use of established ontologies like National Cancer Institute Thesaurus. Each triple should contain a subject-predicate-object relationship which facilitates the structured representation of a clinical trial data. "
# prompt6 = "here is a text with inclusion criteria for a clinical trial. Generate a consistent RDF graph that can be used to create a knowledge graph. "
# Amelie
# prompt7 = "here is a text with inclusion criteria for a clinical trial. Provide an RDF representation which defines classes, properties, relationships, constraints, and hierarchies in a meaningful way."
#prompt8 = "You are an expert expert in Knowledge Graphs. I will give a text with eligibility criteria for a clinical trial and ask you to provide an RDF representation which defines classes, properties, relationships, constraints, and hierarchies in a meaningful way. Make sure to define the prefixes. Here is the text:"
#prompt9 = "You are an expert expert in Knowledge Graphs. I will give a text with eligibility criteria for a clinical trial. Can you provide an RDF representation in Turtle syntax of this text? Make sure to distinguish properly between inclusion and exclusion criteria. Make use of relevant ontologies to present the medical data."
prompt10 = """You are a semantic web specialist with expertise in healthcare and biomedical informatics.
            I will give you a text with eligibility criteria for a clinical trial and ask you to provide an RDF representation of that text.
            You possess the technical proficiency and domain knowledge necessary to interpret the eligibility criteria text effectively.
            Your understanding of medical terminology, clinical trial protocols, and regulatory requirements equips you to extract relevant information accurately from the text.
            Your expertise in RDF and semantic web technologies enables you to transform the extracted information into a structured RDF graph systematically.
            You are adept at mapping entities, relationships, and attributes from the text into RDF triples ensuring semantic consistency and interoperability of the representation.
            Furthermore, your background in healthcare or biomedical informatics provides you with valuable domain-specific knowledge of clinical trials.
            You are familiar with common eligibility criteria patterns, regulatory guidelines, and industry standards, allowing you to interpret the text comprehensively and represent it accurately in RDF format.
            Your attention to detail and analytical skills enable you to handle variations in language, formatting, and expression commonly found in clinical trial documentation.
            You ensure the RDF representation faithfully captures the semantics and nuances of the eligibility criteria text, maintaining precision and reliability throughout the process."""
prompt11 = """I will provide you with a text of eligibility criteria of a clinical trial.
You are a semantic web specialist with expertise in healthcare and biomedical informatics.
You must provide an RDF representation of this text. I am going to tip 500$ for a good solution."""
prompts = [prompt11]
#prompts= [prompt1]

In [ ]:
! pip install hqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.9 MB/s eta 0:00:00
  Created wheel for hqq: filename=hqq-0.1.7.post2-py3-none-any.whl size=58544 sha256=db869c80368bd697cca9b3a5ff9f8edc42d731b6ec6a397233810fba143b5dc5
  Stored in directory: /root/.cache/pip/wheels/a8/54/57/c44b0e431b57bd83b072f4f139ffe030edba811b9817f3e019
Successfully built hqq


In [ ]:
!pip install src

  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for src
  Running setup.py clean for src
Failed to build src
ERROR: Could not build wheels for src, which is required to install pyproject.toml-based projects


In [ ]:
# fix numpy in colab
import numpy
from IPython.display import clear_output

# fix triton in colab
!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
!ldconfig /usr/lib64-nvidia

!git clone https://github.com/dvmazur/mixtral-offloading.git --quiet
!cd mixtral-offloading && pip install -q -r requirements.txt
!huggingface-cli download lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo --quiet --local-dir Mixtral-8x7B-Instruct-v0.1-offloading-demo

clear_output()

In [ ]:
import sys

sys.path.append("mixtral-offloading")
import torch
from torch.nn import functional as F
from hqq.core.quantize import BaseQuantizeConfig
from huggingface_hub import snapshot_download
from IPython.display import clear_output
from tqdm.auto import trange
from transformers import AutoConfig, AutoTokenizer
from transformers.utils import logging as hf_logging

from src.build_model import OffloadConfig, QuantConfig, build_model

hqq_aten package not installed. HQQBackend.ATEN backend will not work unless you install the hqq_aten lib in hqq/kernels.


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
from huggingface_hub import login
login()


In [ ]:

model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
quantized_model_name = "lavawolfiee/Mixtral-8x7B-Instruct-v0.1-offloading-demo"
state_path = "Mixtral-8x7B-Instruct-v0.1-offloading-demo"

config = AutoConfig.from_pretrained(quantized_model_name)

device = torch.device("cuda:0")

##### Change this to 5 if you have only 12 GB of GPU VRAM #####
offload_per_layer = 4
# offload_per_layer = 5
###############################################################

num_experts = config.num_local_experts

offload_config = OffloadConfig(
    main_size=config.num_hidden_layers * (num_experts - offload_per_layer),
    offload_size=config.num_hidden_layers * offload_per_layer,
    buffer_size=4,
    offload_per_layer=offload_per_layer,
)


attn_config = BaseQuantizeConfig(
    nbits=4,
    group_size=64,
    quant_zero=True,
    quant_scale=True,
)
attn_config["scale_quant_params"]["group_size"] = 256


ffn_config = BaseQuantizeConfig(
    nbits=2,
    group_size=16,
    quant_zero=True,
    quant_scale=True,
)
quant_config = QuantConfig(ffn_config=ffn_config, attn_config=attn_config)


model = build_model(
    device=device,
    quant_config=quant_config,
    offload_config=offload_config,
    state_path=state_path,
)

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Loading experts:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
from transformers import TextStreamer


tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
past_key_values = None
sequence = None

seq_len = 0

for t in trial_xml_files:
    print('############################################')
    #t = path+t
    inclusion = get_inclusion_criteria(t)
    exclusion = get_exclusion_criteria(t)
    trial = inclusion + exclusion
    if trial:
      counter = 8
      for p in prompts:
        print(f"User (prompt {counter}): ", end="")
        user_input = f'{p}: {trial}'
        print("\n")

        user_entry = dict(role="user", content=user_input)
        input_ids = tokenizer.apply_chat_template([user_entry], return_tensors="pt").to(device)

        if past_key_values is None:
          attention_mask = torch.ones_like(input_ids)
        else:
          seq_len = input_ids.size(1) + past_key_values[0][0][0].size(1)
          attention_mask = torch.ones([1, seq_len - 1], dtype=torch.int, device=device)

        print(f"Mixtral (prompt {counter}): ", end="")
        result = model.generate(
          input_ids=input_ids,
          attention_mask=attention_mask,
          past_key_values=past_key_values,
          streamer=streamer,
          do_sample=True,
          temperature=0.9,
          top_p=0.9,
          max_new_tokens=1512,
          pad_token_id=tokenizer.eos_token_id,
          return_dict_in_generate=True,
          output_hidden_states=True,
        )
        print("\n")

        sequence = result["sequences"]
        #past_key_values = result["past_key_values"]
        counter += 1

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

############################################
User (prompt 8): 

Mixtral (prompt 8): Here is the RDF representation of the text:

<Criteria>

<EligibilityCriterion>
statement: "Subjects who are willing to participate in the study and sign the informed consent form (ICF) prior to any procedure."
hasVersion: "ICF"
</EligibilityCriterion>

<EligibilityCriterion>
hasAgeRestriction: "≥18 years old"
</EligibilityCriterion>

<EligibilityCriterion>
hasGenderConstraint: "Male or Female"
</EligibilityCriterion>

<EligibilityCriterion>
hasECOGPerformanceStatus: "0-1"
</EligibilityCriterion>

<EligibilityCriterion>
hasLifeExpectancy: "≥ 3 months"
</EligibilityCriterion>

<EligibilityCriterion>
hasMedicalCondition: "Subjects must have histological or cytological confirmation of recurrent or refractory advanced solid tumors, and have progressed on standard therapy"
</EligibilityCriterion>

<EligibilityCriterion>
hasAtLeastOneLesion: "At least one evaluable lesion"
</EligibilityCriterion>

<Eligibilit

KeyboardInterrupt: 